# Anomaly Detection

In this notebook, you get a baseline for anomaly detection.

<img src="https://box.hu-berlin.de/f/53a91798173c4dad9345/?dl=1" width=800/>


### Required Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import plotly.graph_objects as go
import os
import fnmatch
import zipfile

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import scipy as sp

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

### Utility functions

In [2]:
def read_series(path, file, locations=None):   
    print (path + "/"+ file)
    data = pd.read_csv(path + "/"+ file, header=None)
    data = np.array(data).flatten()
    
    # Extract file name
    file_name = file.split('.')[0]
    splits = file_name.split('_')
    test_start = np.array(splits[-1])

    # load the anomalies
    if locations is None:
        locations = pd.read_csv("phase_2/labels.csv")
        locations.set_index("Name", inplace=True)

    # Extract anomaly location
    anomaly = [-1, -1]
    if file_name in locations.index:
        row = locations.loc[file_name]
        anomaly = row["Pos"]

    return (file_name, int(test_start), data, anomaly)


def find_dominant_window_sizes(ts, n_size=1):
    fourier = np.absolute(np.fft.fft(ts))
    freq = np.fft.fftfreq(ts.shape[0], 1)
    coefs = []
    window_sizes = []
    for coef, freq in zip(fourier, freq):
        if coef > 0 and freq > 0 and freq < 0.2:
            window_size = 1.0 / freq
            # avoid too large windows
            if (window_size < 500) and (window_size > 10):
                coefs.append(coef)
                window_sizes.append(window_size)
    coefs = np.array(coefs)
    window_sizes = np.asarray(window_sizes, dtype=np.int64)
    idx = np.argsort(coefs)[::-1]
    
    unique_window_sizes = set()
    for window_size in window_sizes[idx]:
        if len(unique_window_sizes) == n_size:
            return np.array([w for w in unique_window_sizes])
        unique_window_sizes.add(window_size)
    return np.array(list(unique_window_sizes))


def sliding_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


def visualize_with_anomaly_score(
        data, score, test_start, 
        predicted, anomaly, name = None
        ):
    '''Input:
       data: array with the raw data
       test_start: the offset where the test data starts
       predicted: The offset of your prediction.
       anomaly: The offset of the anomaly. 
                      If -1 is passed, no anomaly is plottet.
    '''
    
    anomaly_start = anomaly - 50    
    anomaly_end = anomaly + 50
    predicted_start = predicted - 50
    predicted_end = predicted + 50
    
    fig, ax = plt.subplots(2,1, figsize=(20, 4), sharex=True)
    sns.lineplot(x=np.arange(test_start, len(data)), y=data[test_start:], ax = ax[0], lw=0.5, label="Test")
    sns.lineplot(x=np.arange(0, test_start), y=data[:test_start], ax = ax[0], lw=0.5, label="Train")
        
    if (anomaly_start > 0):
        sns.lineplot(x=np.arange(anomaly_start, anomaly_end), 
                     y=data[anomaly_start:anomaly_end], ax = ax[0], label="Actual")
    
    sns.lineplot(x=np.arange(len(score)), y=score, ax = ax[1], label="Anomaly Scores")
    sns.lineplot(x=np.arange(predicted_start, predicted_end), 
                 y=data[predicted_start:predicted_end], ax = ax[0], label="Predicted")

    if name is not None:
        ax[0].set_title(name)
        
    sns.despine()

    
    #################
    
    plt.legend()
    plt.show()

<hr/>

# Baseline: Anomaly Detection using Predictive Modelling

You are now given an algorithm that computes an anomaly score for each time series:

<img src="https://box.hu-berlin.de/f/2812ccfbcae24e318f9e/?dl=1" width=800/>

The score's maximum indicates where the anomaly is.

In [3]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

#trial file to see what sticks. not much so far tbh.

locations = pd.read_csv("phase_2/labels.csv")
locations.set_index("Name", inplace=True)
anomalie_files = "phase_2"

total_score = 0
predictions = []
ids = []
i = 1

# Distance-based anomaly scores
# Using a k-NN classifier
def distance_based_score(X, test_start):
    clf = KNeighborsClassifier(n_neighbors=5)    
    clf.fit(X, np.ones(len(X)))
    dists, _ = clf.kneighbors(X[test_start:], return_distance=True)        
    
    score = np.zeros(len(X))
    score[test_start:] = dists[:,-1]
    predicted = test_start + np.argmax(score[test_start:])    
    peaks, _ = sp.signal.find_peaks(score, distance=50)
    confidence = np.max(score[peaks])/np.max(score[peaks][score[peaks] != np.max(score[peaks])])
    return score, predicted, confidence
        
    
# Use a classification approach to detect anaomalies
def classifcation_based_scores(X, test_start):   
    clf = LocalOutlierFactor(n_neighbors=5, metric='l2', n_jobs=-1)
    clf.fit_predict(X)    
    score = clf.negative_outlier_factor_
    
    #clf = IsolationForest().fit(X)
    #score = clf.score_samples(X)
    
    #clf = OneClassSVM(nu = 0.01).fit(X)
    #score = clf.score_samples(X)
    predicted = test_start + np.argmin(score[test_start:])    
    peaks, _ = sp.signal.find_peaks(np.negative(score), distance=50)
    confidence = np.min(score[peaks])/np.min(score[peaks][score[peaks] != np.min(score[peaks])])
    return score, predicted, confidence

# Use a regression-based approach to detect anaomalies
def regression_based_scores(X, test_start):
    score = np.zeros(len(X))
    transform = PolynomialFeatures(degree=3)
    
    # generate polynomial features
    XX = transform.fit_transform(X)

    for i, window in enumerate(XX[test_start:]):        
        x = np.arange(len(window)).reshape(-1, 1)
        y = window.reshape(-1, 1)
        
        # train polynomial regression        
        clf = Ridge(normalize=True).fit(x, y)
        pred = clf.predict(x)        
        
        # check error in regression
        score[test_start+i] = np.sum(np.abs(pred - y))
        
        predicted = test_start + np.argmax(score[test_start:])        
    return score, predicted
        
        
        
# Use a forecasting-based approach to detect anaomalies
def forecasting_based_scores(X, test_start):
    score = np.zeros(len(X))
    transform = PolynomialFeatures(degree=2)
    
    # generate polynomial features
    XX = transform.fit_transform(X)

    # learn to predict last value        
    # train polynomial regression        
    xs = XX[:, :-1]
    ys = XX[:, -1] # predict the last value
    clf = Ridge(normalize=True).fit(xs, ys)
    
    # check error in predicting last value
    pred = clf.predict(xs)
    score = np.abs(pred - ys)
        
    predicted = test_start + np.argmax(score[test_start:])        
    peaks, _ = sp.signal.find_peaks(score, distance=50)
    confidence = np.max(score[peaks])/np.max(score[peaks][score[peaks] != np.max(score[peaks])])
    return score, predicted, confidence

for file in np.sort(fnmatch.filter(os.listdir(anomalie_files), "*.csv")):
    if "Anomaly" in str(file):
        file_name = file.split('.')[0]
        name, test_start, data, anomaly = read_series(anomalie_files, file, locations)

        periods = find_dominant_window_sizes(data[:test_start])
        window_size = np.int32(periods[0] / 4)
        X = sliding_window(data, window_size)

        #score_dist, predicted_dist, confidence_dist = distance_based_score(X, test_start)
        score_class, predicted_class, confidence_class = classifcation_based_scores(X, test_start)
        #score, predicted = regression_based_scores(X, test_start)        
        #score_fore, predicted_fore, confidence_fore = forecasting_based_scores(X, test_start)
        #if (confidence_dist>confidence_class and confidence_dist>confidence_fore):
        #    score = score_dist
        #    predicted = predicted_dist
        #elif confidence_class > confidence_fore:
        score = score_class
        predicted = predicted_class
        #else:
        #    score = score_fore
        #    predicted = predicted_fore
        
        predictions.append(predicted)
        ids.append(file_name)
        
        score[:test_start] = np.NaN

        # Visualize the predicted anomaly
        #visualize_with_anomaly_score(
        #     data, score, test_start, predicted, anomaly, name)

        if (anomaly > -1):
            total_score += abs(anomaly - predicted) / (anomaly)        
            i = i+1
            
        print("Current Score: ", (total_score / i) * 100)

        
print("\tTotal score:", (total_score / len(locations)) * 100)

phase_2/001_Anomaly_5000.csv
Current Score:  17.92225201072386
phase_2/002_Anomaly_4375.csv
Current Score:  17.966947349872246
phase_2/003_Anomaly_4375.csv
Current Score:  24.440047562146894
phase_2/004_Anomaly_2500.csv
Current Score:  19.83168406741663
phase_2/005_Anomaly_4000.csv
Current Score:  19.81899598210645
phase_2/006_Anomaly_4000.csv
Current Score:  19.473926380652646
phase_2/007_Anomaly_4000.csv
Current Score:  17.072377890763374
phase_2/008_Anomaly_4000.csv
Current Score:  15.179443816569844
phase_2/009_Anomaly_4000.csv
Current Score:  13.712781486194908
phase_2/010_Anomaly_4000.csv
Current Score:  12.481216161441488
phase_2/011_Anomaly_3333.csv
Current Score:  11.578245616342462
phase_2/012_Anomaly_5000.csv
Current Score:  11.466530152345571
phase_2/013_Anomaly_5000.csv
Current Score:  10.650167511715216
phase_2/014_Anomaly_2666.csv
Current Score:  9.944855316367388
phase_2/015_Anomaly_250.csv
Current Score:  9.405147097189664
phase_2/016_Anomaly_1666.csv
Current Score:  8

Current Score:  13.210999514063278
phase_2/131_Anomaly_4274.csv
Current Score:  13.30048495228932
phase_2/132_Anomaly_5059.csv
Current Score:  13.30048495228932
phase_2/133_Anomaly_2246.csv
Current Score:  13.183618775263525
phase_2/134_Anomaly_2272.csv
Current Score:  13.183618775263525
phase_2/135_Anomaly_3272.csv
Current Score:  13.523297264085322
phase_2/136_Anomaly_3872.csv
Current Score:  13.40369822715782
phase_2/137_Anomaly_1752.csv
Current Score:  13.40369822715782
phase_2/138_Anomaly_1752.csv
Current Score:  13.79331643360382
phase_2/139_Anomaly_2101.csv
Current Score:  14.081063580923697
phase_2/140_Anomaly_2101.csv
Current Score:  14.216059275398063
phase_2/141_Anomaly_2101.csv
Current Score:  14.152762524604087
phase_2/142_Anomaly_2145.csv
Current Score:  14.197610708154478
phase_2/143_Anomaly_2145.csv
Current Score:  14.366287306181663
phase_2/144_Anomaly_2367.csv
Current Score:  14.245894789628972
phase_2/145_Anomaly_2135.csv
Current Score:  14.453434501916277
phase_2/14

# Submit your solution to Kaggle

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
Create a submission named `submission.csv` using your model and upload it to kaggle:

- Phase 1: https://www.kaggle.com/c/time-series-anomaly-detection
- Phase 2: https://www.kaggle.com/competitions/time-series-anomaly-detection-phase-2

</div>

In [4]:
# Create a submission
submission = pd.DataFrame({'ID': ids,'PREDICTED': predictions})

#Visualize the first 5 rows
display(submission)

,ID,PREDICTED
0,001_Anomaly_5000,10134
1,002_Anomaly_4375,8382
2,003_Anomaly_4375,8387
3,004_Anomaly_2500,5571
4,005_Anomaly_4000,5389
...,...,...
145,146_Anomaly_2667,9196
146,147_Anomaly_2777,6216
147,148_Anomaly_3571,6732
148,149_Anomaly_4000,6812


In [5]:
filename = 'baseline_submission_phase_2.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: baseline_submission_phase_2.csv


# 